In [1]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver
import os
import re

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘


dart_list = dart.list(start='20241111',end=today)
lists = dart_list[dart_list['report_nm'].str.contains('연결재무제표기준영업')|dart_list['report_nm'].str.contains('잠정')]
lists = lists[~lists['report_nm'].str.contains('기재정정')]
print(f'{len(lists)}개 조회')


# lists = df[(df['report_nm'] == '연결재무제표기준영업(잠정)실적(공정공시)')|(df['report_nm'] == '영업(잠정)실적(공정공시)')]#|        
lists['구분']=lists['report_nm'].str[:2].replace('영업','별도')
lists = lists.sort_values(by=['report_nm'])


ttl_df=pd.DataFrame()

for rcp_no,name,gobun in zip(lists['rcept_no'],lists['flr_nm'],lists['구분']):
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(options=options)
        driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
        driver.switch_to.frame('ifrm')
        html = driver.page_source #지금 현 상태의 page source불러오기
        soup = BeautifulSoup(html,'html.parser')
        tables = soup.find_all('table')#표를 불러옴
        p=parser.make2d(tables[0])#표를 만듬
        tt=pd.DataFrame(p)
        try:
            won = tt[tt[0].str.contains('단위', na=False)][0].iloc[0]
            match = re.search(r'단위\s*:\s*(\w+)', won)
            unit = match.group(1)
        except:
            won = tt[tt[6].str.contains('단위', na=False)][6].iloc[0]
            match = re.search(r'단위\s*:\s*(\w+)', won)
            unit = match.group(1)

        tt = tt[tt[1] == '당해실적']

        tt = tt[~tt[0].str.contains('(법인세|지배)')]
        tt = tt[[0,2,3,5]]
        tt.columns=['항목','24.3Q','24.2Q','23.3Q']
        tt.insert(0,'단위',unit)
        tt.insert(0,'구분',gobun)
        tt.insert(0,'종목명',name)
        ttl_df = pd.concat([ttl_df,tt])
        time.sleep(1)
    except Exception as e:
        print(e)
        

file_name = './24년3분기잠정실적_241001.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
ttl_df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()


# ttl_df.to_excel('24년3분기잠정실적_241001.xlsx',index=False)

61개 조회


C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\L

6


C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\Local\Temp\ipykernel_20060\1211506117.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]
C:\Users\codek\AppData\L

6


In [ ]:
# rcp_no = '20241107800267'
# name = 'GS'
# gobun = '연결'

ttl_df=pd.DataFrame()

for rcp_no,name,gobun in zip(lists['rcept_no'],lists['flr_nm'],lists['구분']):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    try:
        won = tt[tt[0].str.contains('단위', na=False)][0].iloc[0]
        match = re.search(r'단위\s*:\s*(\w+)', won)
        unit = match.group(1)
    except:
        won = tt[tt[6].str.contains('단위', na=False)][6].iloc[0]
        match = re.search(r'단위\s*:\s*(\w+)', won)
        unit = match.group(1)

    tt = tt[tt[1] == '당해실적']

    tt = tt[~tt[0].str.contains('(법인세|지배)')]
    tt = tt[[0,2,3,5,4,6]]
    tt.columns=['항목','24.3Q','24.2Q','23.3Q','QoQ','YoY']
    tt.insert(0,'단위',unit)
    tt.insert(0,'구분',gobun)
    tt.insert(0,'종목명',name)
    ttl_df = pd.concat([ttl_df,tt])


C:\Users\codek\AppData\Local\Temp\ipykernel_12968\3709299303.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]


       종목명  구분   단위     항목   24.3Q   24.2Q   23.3Q              QoQ  \
4   하이록코리아  연결  백만원    매출액  45,285  46,087  46,592     -802 (-1.7%)   
6   하이록코리아  연결  백만원   영업이익  13,656  14,699  14,146   -1,043 (-7.1%)   
10  하이록코리아  연결  백만원  당기순이익   9,150  13,059  12,804  -3,909 (-29.9%)   

                YoY  
4    -1,307 (-2.8%)  
6      -490 (-3.5%)  
10  -3,654 (-28.5%)  


C:\Users\codek\AppData\Local\Temp\ipykernel_12968\3709299303.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]


           종목명  구분   단위     항목   24.3Q   24.2Q   23.3Q              QoQ  \
4   BGF에코머티리얼즈  연결  백만원    매출액  86,533  92,517  65,005  -5,984 (-6.47%)   
6   BGF에코머티리얼즈  연결  백만원   영업이익   3,263   3,634   4,031   -371 (-10.21%)   
10  BGF에코머티리얼즈  연결  백만원  당기순이익   1,876   2,773   3,625   -897 (-32.35%)   

                 YoY  
4    21,528 (33.12%)  
6     -768 (-19.05%)  
10  -1,749 (-48.25%)  


C:\Users\codek\AppData\Local\Temp\ipykernel_12968\3709299303.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]


...

C:\Users\codek\AppData\Local\Temp\ipykernel_12968\3709299303.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]


        종목명  구분   단위     항목   24.3Q    24.2Q    23.3Q               QoQ  \
4   스튜디오드래곤  연결  백만원    매출액  90,320  137,120  217,412  -46,800 (-34.1%)   
6   스튜디오드래곤  연결  백만원   영업이익    -937   10,454   21,853   -11,390 (적자 전환)   
10  스튜디오드래곤  연결  백만원  당기순이익  -6,128    8,152   18,173   -14,280 (적자 전환)   

                  YoY  
4   -127,092 (-58.5%)  
6     -22,789 (적자 전환)  
10    -24,300 (적자 전환)  


C:\Users\codek\AppData\Local\Temp\ipykernel_12968\3709299303.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tt = tt[~tt[0].str.contains('(법인세|지배)')]


       종목명  구분  단위     항목  24.3Q  24.2Q  23.3Q    QoQ    YoY
4   현대그린푸드  연결  억원    매출액  6,016  5,424  5,746  10.92   4.71
6   현대그린푸드  연결  억원   영업이익    383    325    283  17.82  35.32
10  현대그린푸드  연결  억원  당기순이익    305    236    220  29.63  38.97


KeyboardInterrupt: 

In [ ]:
# p
# import re
# won = tt[tt[6].str.contains('단위')][0].iloc[0]
# # match = re.search(r'단위\s*:\s*(\w+)', won)
# # unit = match.group(1)
# won

tt[tt[5].str.contains('단위', na=False)][5].iloc[0]

'단위 : 억원, %'